# Solution

# Задание 2. SHAP. Изображения: FashionMNIST
Дана сверточная нейронная сеть, решающая задачу [FashionMNIST](https://pytorch.org/vision/stable/generated/torchvision.datasets.FashionMNIST.html#torchvision.datasets.FashionMNIST) (Классификация изображений одежды).

Визуализируйте результат SHAP для картинок.

## Формат результата

Визуализация работы SHAP для FashionMNIST с помощью [shap.image_plot](https://shap.readthedocs.io/en/latest/example_notebooks/api_examples/plots/image.htmll)

Пример визуализации:
<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.0/Exercises/EX14/result_2_task_ex14.png" width="1000">

## Модель

Импорт библиотек:

In [ ]:
!pip install -q shap

In [ ]:
import shap
from IPython.display import clear_output

device = "cpu"

Описание модели:

In [ ]:
from torch import nn


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=5),
            nn.MaxPool2d(2),
            nn.LeakyReLU(),
            nn.Conv2d(10, 20, kernel_size=5),
            nn.Dropout(),
            nn.MaxPool2d(2),
            nn.LeakyReLU(),
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(320, 50),
            nn.LeakyReLU(),
            nn.Dropout(),
            nn.Linear(50, 10),  # ten classes
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(-1, 320)
        x = self.fc_layers(x)
        return x

Функции для обучения модели:

In [ ]:
from torch.nn.functional import cross_entropy


def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(data),
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.item(),
                )
            )

Загрузка данных и разбиение на train и test:


In [ ]:
from torchvision.datasets import FashionMNIST
from torchvision import transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([transforms.ToTensor()])

train_ds = FashionMNIST("fm", train=True, download=True, transform=transform)
test_ds = FashionMNIST("fm", train=False, download=True, transform=transform)

batch_size = 128
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

Обучение модели:

In [ ]:
from torch import optim

num_epochs = 2
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)

## Визуализация SHAP

Выбираем из тестовых данных по одному элементу каждого класса (всего 10 классов)

In [ ]:
print("Classes: ", ", ".join(train_ds.classes))

Выберем по одному элементу каждого класса

In [ ]:
import numpy as np

idx = []
for class_num in range(len(train_ds.classes)):
    matched_idx = np.where(test_ds.targets.numpy() == class_num)
    idx.append(matched_idx[0][0])
images = test_ds.data[idx]
print(images.shape)

Убедимся, что картинки действительно из разных классов:

In [ ]:
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

grid_img = make_grid(images.unsqueeze(1), nrow=10)
plt.imshow(grid_img.permute(1, 2, 0))
plt.show()

Теперь инициализируем [shap.DeepExplainer](https://shap-lrjball.readthedocs.io/en/latest/generated/shap.DeepExplainer.html) так, как мы делали в лекции:

In [ ]:
import torch

# Get background for DeepLift https://arxiv.org/abs/1704.02685
# 1. generate 1000 random indexes
inds = torch.randint(0, train_ds.data.shape[0], size=(1000,))
# 2. Using indexes extract 1000 samples from dataset
# because original data in uint8, convert it to float
# and add channel dimension [1000, 28, 28] -> [1000, 1, 28, 28]
background = train_ds.data[inds].float().unsqueeze(1)
explainer = shap.DeepExplainer(model, background)

Применяем SHAP к выбранным нами 10 случайным изображениям*.

*Попиксельный SHAP очень тяжелый!

In [ ]:
from warnings import simplefilter

simplefilter("ignore", category=Warning)

# convert to float and add channel dimension [10, 28, 28] -> [10, 1, 28, 28]
test_images = images.unsqueeze(1).float()
shap_values = explainer.shap_values(test_images)  # shap magic here

Получили значения Шепли для каждого пикселя каждой картинки:

In [ ]:
print(len(shap_values))
print(shap_values[0].shape)
print(test_images.shape)

Задание: Визуализируйте результат работы SHAP, используя [shap.image_plot](https://shap.readthedocs.io/en/latest/generated/shap.plots.image.html?highlight=image_plot)

In [ ]:
# Your code here
# shape of image for shap = [batch, 28, 28, 1]

shap_list_numpy = []  # list of np.array

for v in shap_values:
    # because image_plot accept data in form(#samples x width x height x channels)
    # move cannel to last dimension [10, 1, 28, 28] -> (10,  28, 28, 1)
    shap_list_numpy.append(np.moveaxis(v, (1), (3)))

    # Do the same for original images in pytorch format and convert it to numpy
    test_numpy = test_images.permute(0, 2, 3, 1).numpy()

# End of your code

# Generate a matrix of labels for all cell. Because we want labels only in head
# rest of table filled with  empty strings
labels = np.full((10, 10), " " * 20)
labels[0] = train_ds.classes

# plot the feature attributions
shap.image_plot(shap_list_numpy, -test_numpy, labels=labels)

Какие характерные особенности элементов одежды выделяет SHAP? (можно рассмотреть на примере T-shirt и Trouser).

**Напишите вывод:**

## Памятка для преподавателя

Цель задания — показать, что SHAP может работать с изображениями.

Частая ошибка: shap_list_numpy для визуализации должен быть листом np.array, иначе SHAP его не съест.

В выводе студент должен написать, что SHAP позволил выделить особенности элементов одежды, например, для штанов это пространство между штанинами, для футболки — короткие рукава.

За отсутствие вывода баллы снижаются.

# Версия для студентов

# Задание 2. SHAP. Изображения: FashionMNIST
Дана сверточная нейронная сеть, решающая задачу [FashionMNIST](https://pytorch.org/vision/stable/generated/torchvision.datasets.FashionMNIST.html#torchvision.datasets.FashionMNIST) (Классификация изображений одежды).

Визуализируйте результат SHAP для картинок.

## Формат результата

Визуализация работы SHAP для FashionMNIST с помощью [shap.image_plot](https://shap.readthedocs.io/en/latest/example_notebooks/api_examples/plots/image.htmll)

Пример визуализации:
<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.0/Exercises/EX14/result_2_task_ex14.png" width="1000">

## Модель

Импорт библиотек:

In [ ]:
!pip install -q shap

In [ ]:
import shap
from IPython.display import clear_output

device = "cpu"

Описание модели:

In [ ]:
from torch import nn


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=5),
            nn.MaxPool2d(2),
            nn.LeakyReLU(),
            nn.Conv2d(10, 20, kernel_size=5),
            nn.Dropout(),
            nn.MaxPool2d(2),
            nn.LeakyReLU(),
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(320, 50),
            nn.LeakyReLU(),
            nn.Dropout(),
            nn.Linear(50, 10),  # ten classes
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(-1, 320)
        x = self.fc_layers(x)
        return x

Функции для обучения модели:

In [ ]:
from torch.nn.functional import cross_entropy


def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(data),
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.item(),
                )
            )

Загрузка данных и разбиение на train и test:


In [ ]:
from torchvision.datasets import FashionMNIST
from torchvision import transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([transforms.ToTensor()])

train_ds = FashionMNIST("fm", train=True, download=True, transform=transform)
test_ds = FashionMNIST("fm", train=False, download=True, transform=transform)

batch_size = 128
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

Обучение модели:

In [ ]:
from torch import optim

num_epochs = 2
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)

## Визуализация SHAP

Выбираем из тестовых данных по одному элементу каждого класса  (всего 10 классов)

In [ ]:
print("Classes: ", ", ".join(train_ds.classes))

Выберем по одному элементу каждого класса

In [ ]:
import numpy as np

idx = []
for class_num in range(len(train_ds.classes)):
    matched_idx = np.where(test_ds.targets.numpy() == class_num)
    idx.append(matched_idx[0][0])
images = test_ds.data[idx]
print(images.shape)

Убедимся, что картинки действительно из разных классов:

In [ ]:
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

grid_img = make_grid(images.unsqueeze(1), nrow=10)
plt.imshow(grid_img.permute(1, 2, 0))
plt.show()

Теперь инициализируем [shap.DeepExplainer](https://shap-lrjball.readthedocs.io/en/latest/generated/shap.DeepExplainer.html) так, как мы делали в лекции:

In [ ]:
import torch

# Get background for DeepLift https://arxiv.org/abs/1704.02685
# 1. generate 1000 random indexes
inds = torch.randint(0, train_ds.data.shape[0], size=(1000,))
# 2. Using indexes extract 1000 samples from dataset
# because original data in uint8, convert it to float
# and add channel dimension [1000, 28, 28] -> [1000, 1, 28, 28]
background = train_ds.data[inds].float().unsqueeze(1)
explainer = shap.DeepExplainer(model, background)

Применяем SHAP к выбранным нами 10 случайным изображениям*.

*Попиксельный SHAP очень тяжелый!

In [ ]:
from warnings import simplefilter

simplefilter("ignore", category=Warning)

# convert to float and add channel dimension [10, 28, 28] -> [10, 1, 28, 28]
test_images = images.unsqueeze(1).float()
shap_values = explainer.shap_values(test_images)  # shap magic here

Получили значения Шепли для каждого пикселя каждой картинки:

In [ ]:
print(len(shap_values))
print(shap_values[0].shape)
print(test_images.shape)

Задание: Визуализируйте результат работы SHAP, используя [shap.image_plot](https://shap.readthedocs.io/en/latest/generated/shap.plots.image.html?highlight=image_plot)

In [ ]:
# shape of image for shap = [batch, 28, 28, 1]

# Your code here
shap_list_numpy = ...  # list of np.array
test_numpy = ...

# End of your code

# Generate a matrix of labels for all cell. Because we want labels only in head
# rest of table filled with  empty strings
labels = np.full((10, 10), " " * 20)
labels[0] = train_ds.classes

# plot the feature attributions
shap.image_plot(shap_list_numpy, -test_numpy, labels=labels)

Какие характерные особенности элементов одежды выделяет SHAP? (можно рассмотреть на примере T-shirt и Trouser).

**Напишите вывод:**